In [1]:
import sys
spath = '../'
sys.path.insert(0,spath)

In [2]:
from scripts.models import get_rttm_segments_features

In [3]:
development_rttm     = '../exp/dihardii/development/ref_augmented_0_1.5_0.5_0.5.rttm'
development_segments = '../exp/dihardii/development/augmented_0/segments'
development_ivectors = '../exp/dihardii/development/augmented_0/exp/make_ivectors/ivector.txt'
development_files_segments = get_rttm_segments_features(development_rttm, development_segments, development_ivectors)

In [ ]:
evaluation_rttm     = '../exp/dihardii/evaluation/ref_augmented_0_1.5_0.5_0.5.rttm'
evaluation_segments = '../exp/dihardii/evaluation/augmented_0/segments'
evaluation_ivectors = '../exp/dihardii/evaluation/augmented_0/exp/make_ivectors/ivector.txt'
evaluation_files_segments = get_rttm_segments_features(evaluation_rttm, evaluation_segments, evaluation_ivectors)